# Mitigation Visualizations

## Set up

In [1]:
import numpy as np
import pandas as pd
import altair as alt

In [2]:
def print_column_info(df, col_name):
    """
    Print range, completeness, and data type of column with provided name and dataframe
    """
    print(f'COLUMN {col_name} | Range: {df[col_name].min()} to {df[col_name].max()} | NAs: {df[col_name].isna().sum()} | Type: {df[col_name].dtype}')

In [3]:
# load data
df = pd.read_csv('../../data/raw/rawdata_new.csv')

## Survey Overview

These visualizations are related to two questions on the survey:
* *Q5*: Which of the following measures do you currently implement on your farm and do you consider the measures effective for climate change mitigation?
* *Q6*: Which of the measures that you do not currently implement could you imagine to adopt in the future, which not?

These correspond to many features in the dataset. For each of the 13 mitigations, there are three columns. For example, the mitigation measure *"I keep my cows for at least 5 lactation periods."* has three related features:
* `breed` refers to whether the farmer has implemented the measure
    - 1=yes, 0=no
    - NA=not relevant
* `breed_eff` refers to whether the farmer considers the measure effective
    - 1=not effective at all, 2, 3, 4, 5=very effective
    - NA=don't know
* `breed_imag` refers to whether the farmer can imagine implementing the measure
    - 1=I can imagine to implement it, 0=I cannot imagine to implement it
    - NA=not relevant or already implemented

## Data transformation

### Quick sanity check
Are the mitigations implemented columns the size, type and range expected?

In [4]:
# define list of mitigation implemented columns
implemented_columns = ['legum', 'conc', 'add', 'lact', 'breed', 'covman', 'comp', 'drag', 'cov', 'plough', 'solar', 'biog', 'ecodr']
# quick sanity check
for col in implemented_columns:
    print_column_info(df, col)

COLUMN legum | Range: 1 to 3 | NAs: 0 | Type: int64
COLUMN conc | Range: 1 to 3 | NAs: 0 | Type: int64
COLUMN add | Range: 1 to 3 | NAs: 0 | Type: int64
COLUMN lact | Range: 1 to 3 | NAs: 0 | Type: int64
COLUMN breed | Range: 1 to 3 | NAs: 0 | Type: int64
COLUMN covman | Range: 1 to 3 | NAs: 0 | Type: int64
COLUMN comp | Range: 1 to 3 | NAs: 0 | Type: int64
COLUMN drag | Range: 1 to 2 | NAs: 0 | Type: int64
COLUMN cov | Range: 1 to 3 | NAs: 0 | Type: int64
COLUMN plough | Range: 1 to 3 | NAs: 0 | Type: int64
COLUMN solar | Range: 1 to 2 | NAs: 0 | Type: int64
COLUMN biog | Range: 2 to 2 | NAs: 0 | Type: int64
COLUMN ecodr | Range: 1 to 2 | NAs: 0 | Type: int64


Ok, well here is a surprise. The codebook says these measures will have values of 0, 1, and NA; instead, they have values of 1, 2, and 3. I'll see if I can figure out which values are which based on some exploration and (of course) assumptions. My bet is that farms with no animal units reported will consider the livestock mitigations not applicable.

In [5]:
# define list of mitigation effective columns
effective_columns = [c for c in df.columns if '_eff' in c]
# quick sanity check
for col in effective_columns:
    print_column_info(df, col)

COLUMN legum_eff | Range: 1.0 to 5.0 | NAs: 20 | Type: float64
COLUMN conc_eff | Range: 1.0 to 5.0 | NAs: 20 | Type: float64
COLUMN add_eff | Range: 1.0 to 5.0 | NAs: 31 | Type: float64
COLUMN lact_eff | Range: 1.0 to 5.0 | NAs: 29 | Type: float64
COLUMN breed_eff | Range: 1.0 to 4.0 | NAs: 44 | Type: float64
COLUMN covman_eff | Range: 1.0 to 5.0 | NAs: 19 | Type: float64
COLUMN comp_eff | Range: 1.0 to 5.0 | NAs: 30 | Type: float64
COLUMN drag_eff | Range: 1.0 to 5.0 | NAs: 12 | Type: float64
COLUMN cov_eff | Range: 1.0 to 5.0 | NAs: 13 | Type: float64
COLUMN plough_eff | Range: 1.0 to 5.0 | NAs: 10 | Type: float64
COLUMN solar_eff | Range: 1.0 to 5.0 | NAs: 16 | Type: float64
COLUMN biog_eff | Range: 1.0 to 5.0 | NAs: 26 | Type: float64
COLUMN ecodr_eff | Range: 1.0 to 5.0 | NAs: 9 | Type: float64


Lovely, these are in the expected range, completeness, and data type.

In [6]:
# define list of mitigation imaginable columns
imaginable_columns = [c for c in df.columns if '_imag' in c]
# quick sanity check
for col in imaginable_columns:
    print_column_info(df, col)

COLUMN legum_imag | Range: 0.0 to 1.0 | NAs: 84 | Type: float64
COLUMN conc_imag | Range: 0.0 to 1.0 | NAs: 83 | Type: float64
COLUMN add_imag | Range: 0.0 to 1.0 | NAs: 90 | Type: float64
COLUMN lact_imag | Range: 0.0 to 1.0 | NAs: 80 | Type: float64
COLUMN breed_imag | Range: 0.0 to 1.0 | NAs: 52 | Type: float64
COLUMN covman_imag | Range: 0.0 to 1.0 | NAs: 91 | Type: float64
COLUMN comp_imag | Range: 0.0 to 1.0 | NAs: 50 | Type: float64
COLUMN drag_imag | Range: 0.0 to 1.0 | NAs: 86 | Type: float64
COLUMN cov_imag | Range: 0.0 to 1.0 | NAs: 67 | Type: float64
COLUMN plough_imag | Range: 0.0 to 1.0 | NAs: 33 | Type: float64
COLUMN solar_imag | Range: 0.0 to 1.0 | NAs: 27 | Type: float64
COLUMN biog_imag | Range: 0.0 to 1.0 | NAs: 32 | Type: float64
COLUMN ecodr_imag | Range: 0.0 to 1.0 | NAs: 79 | Type: float64


Great, these are also in the expected range, completeness and data type.

### Create a mitigation dataframe

In [7]:
# convert implemented mitigations from wide to long format
implemented_df = pd.melt(df, id_vars='id', value_vars=implemented_columns, var_name='mitigation', value_name='implemented')
# set id and mitigation as index
implemented_df.set_index(['id', 'mitigation'], inplace=True)
# quick peek
implemented_df.sample(5)

,,implemented
id,mitigation,
96,comp,2
84,solar,2
93,conc,2
97,ecodr,1
22,add,2


In [8]:
# convert effective mitigations from wide to long format
effective_df = pd.melt(df, id_vars='id', value_vars=effective_columns, var_name='mitigation', value_name='effective')
# remove suffix from mitigation name
effective_df['mitigation'] = [s[0:-len('_eff')] for s in effective_df['mitigation']]
# set id and mitigation as index
effective_df.set_index(['id', 'mitigation'], inplace=True)
# quick peek
effective_df.sample(5)

,,effective
id,mitigation,
80,biog,1.0
35,conc,5.0
94,plough,3.0
76,biog,5.0
97,comp,3.0


In [9]:
# convert imaginable mitigations from wide to long format
imaginable_df = pd.melt(df, id_vars='id', value_vars=imaginable_columns, var_name='mitigation', value_name='imaginable')
# remove suffix from mitigation name
imaginable_df['mitigation'] = [s[0:-len('_imag')] for s in imaginable_df['mitigation']]
# set id and mitigation as index
imaginable_df.set_index(['id', 'mitigation'], inplace=True)
# quick peek
imaginable_df.sample(5)

,,imaginable
id,mitigation,
60,solar,0.0
102,ecodr,NaN
104,drag,NaN
16,biog,1.0
2,cov,NaN


In [10]:
# create farm type and animal unit dataframe
farm_df = df[['id', 'farmtype']]
# set id as index
farm_df.set_index('id', inplace=True)

In [14]:
# join dataframes together
mitigation_df = implemented_df.copy()
mitigation_df = mitigation_df.join(effective_df)
mitigation_df = mitigation_df.join(imaginable_df)
mitigation_df = mitigation_df.join(farm_df, on='id')
# reset index for altair
mitigation_df.reset_index(inplace=True)
# quick peek
mitigation_df.head()

,id,mitigation,implemented,effective,imaginable,farmtype,animal_unit
0,1,legum,2,NaN,NaN,Livestock,3.9250
1,2,legum,3,NaN,NaN,Arable farming,0.0000
2,3,legum,2,4.0,1.0,Livestock,15.9528
3,4,legum,2,3.0,1.0,Livestock,52.9000
4,5,legum,2,3.0,NaN,Livestock,56.5108


### Add features to dataframe

In [ ]:
# TODO

## Visualizations

### Do implemented mitigations differ by farm type?
And what does that tell me about which values are which for mitigation?

In [20]:
alt.Chart(mitigation_df).mark_bar().encode(
    x='count(implemented)',
    y=alt.Y('mitigation', sort=implemented_columns),
    color='implemented:N',
    column='farmtype'
)

alt.Chart(...)

I am going to assume the following:
* 3 (pink) = not applicable
* 2 (orange) = no, not implemented
* 1 (blue) = yes, implemented

I am assuming this based on the fact that the arable farming farms will not be able to implement livestock mitigations, so they will mostly answer not applicable followed by no not implemented. 

### What mitigations are implemented most and least often?

In [29]:
alt.Chart(mitigation_df).mark_bar().encode(
    x='count(implemented)',
    y=alt.Y('mitigation', sort=implemented_columns),
    color='implemented:N'
)

alt.Chart(...)

The ecodrive, draghose, cover crops, and cover manure mitigations are most implemented - over half of the farms surveyed have implemented these mitigations. The remaining mitigations are less popular, and about one third or fewer of farmers surveyed have implemented those mitigations on their farm. Biogas is the least popular, with 0 farmers surveyed having implemented it.

### What mitigations do farmers see as effective?
Reminder that 1=not effective at all, 2, 3, 4, 5=very effective and NA=don't know.

In [31]:
alt.Chart(mitigation_df).mark_bar().encode(
    x='count(effective)',
    y=alt.Y('mitigation', sort=implemented_columns),
    color='effective:O'
)

alt.Chart(...)

Draghose, solar, ecodrive, cover manure and legumes had the most very effective (5) and effective (4) ratings. Breed, additives, compost, and biogas had the most 'I don't know' (null) ratings.

### Does perceived effectiveness of mitigations differ by farm type?

In [27]:
alt.Chart(mitigation_df).mark_bar().encode(
    x='count(effective)',
    y=alt.Y('mitigation', sort=implemented_columns),
    color='effective:O',
    column='farmtype'
)

alt.Chart(...)

Unsurprisingly, arable farming farmers responded 'I don't know' about the livestock and manure management measures the most. Biogas was also often rated as 'I don't know' about its effectiveness. The livestock farmers were most unfamiliar with the effectiveness of breed.

### What mitigations can farmers imagine implementing?

In [32]:
alt.Chart(mitigation_df).mark_bar().encode(
    x='count(imaginable)',
    y=alt.Y('mitigation', sort=implemented_columns),
    color='imaginable:O'
)

alt.Chart(...)

Most farmers can imagine implementing solar. Fewest farmers can imagine implementing manure coverage.

### Does knowledge of mitigations differ by farm type?

In [33]:
alt.Chart(mitigation_df).mark_bar().encode(
    x='count(imaginable)',
    y=alt.Y('mitigation', sort=implemented_columns),
    color='imaginable:O',
    column='farmtype'
)

alt.Chart(...)

Arable farming farmers without the measures already can imagine implementing solar and cover crops more than other measures. Livestock farmers without the measures already can imagine implementing breed, solar and biogas.

### How many farms are implementing as many mitigations as possible?

### Which farms are implementing the highest proportion of mitigations possible?